In [5]:
%load_ext dotenv
%dotenv

In [8]:

import torch
from torch import nn
from torch import optim

from torch.utils.data import DataLoader

from model import Lilly

from dataset import GSM8KDataset
from datasets import load_dataset

from tqdm.contrib.telegram import tqdm, trange

In [3]:
dataset = load_dataset("gsm8k", "main")

train_data = dataset['train'] # pyright: ignore[reportIndexIssue]
gsm_dataset = GSM8KDataset(train_data)

batch_size = 32  # Smaller batch size to fit into memory

data_loader = DataLoader(gsm_dataset, batch_size=batch_size, shuffle=True)

Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 417310.63 examples/s]


In [6]:
# Hyperparameters

embedding_dim = 64
nhead = 4
num_layers = 2
dim_feedforward = 128
epochs = 5
learning_rate = 0.001

# Instantiate the model
model = Lilly(
    vocab_size=gsm_dataset.vocab_size,
    embedding_dim=embedding_dim,
    nhead=nhead,
    num_layers=num_layers,
    dim_feedforward=dim_feedforward,
    token2idx=gsm_dataset.token2idx
)

# Move model to device
device_id = (
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

device = torch.device(device_id)
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate) # pyright: ignore [reportPrivateImportUsage]

def train_model(model, data_loader, epochs):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        progress_bar = tqdm(enumerate(data_loader), total=len(data_loader), desc=f"Epoch {epoch+1}/{epochs}")
        for batch_idx, (data, target) in progress_bar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            output = model(data)
            loss = criterion(output.view(-1, gsm_dataset.vocab_size), target.view(-1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            progress_bar.set_postfix(loss=loss.item())
            # if batch_idx % 100 == 0:
            #     print(f'Epoch [{epoch+1}/{epochs}], Step [{batch_idx}/{len(data_loader)}], Loss: {loss.item():.4f}')
        print(f'Epoch [{epoch+1}/{epochs}] completed with average loss: {epoch_loss / len(data_loader):.4f}')


/Users/gent/.pyenv/versions/3.12.1/envs/lilly/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:
train_model(model, data_loader, 1)

Epoch 1/1:   4%|▎         | 1993/54431 [32:36<14:18:00,  1.02it/s, loss=2.43]


KeyboardInterrupt: 